In [1]:
import os  
import numpy as np
import operator
import sys

#图片的大小
IMAGE_ROW = 28
IMAGE_COL = 28
IMAGE_SIZE = 28*28

In [2]:
'''
功能:
获取文件头dimension数据
入参：
filename, 文件名称
返回：
返回文件头的dimension数据
'''
def read_head(filename):
    print('读取文件头：',os.path.basename(filename))
    dimension = []
    with open(filename,'rb') as pf:
        #获取magic number
        data = pf.read(4)#读出第1个4字节
        magic_num = int.from_bytes(data,byteorder='big')#bytes数据大尾端模式转换为int型
        print('magicnum: ', hex(magic_num))

        #获取dimension的长度，由magic number的最后一个字节确定
        dimension_cnt = magic_num & 0xff
        
        #获取dimension数据，
        #dimension[0]表示图片的个数,如果是3维数据,dimension[1][2]分别表示其行/列数值
        for i in range(dimension_cnt):
            data = pf.read(4)
            dms = int.from_bytes(data,byteorder='big')
            print('dimension %d: %d'%(i,dms))
            dimension.append(dms)
    print(dimension)
    return dimension

In [3]:
'''
功能:
文件头的长度为4字节的magic num+dimension的个数*4
入参：
dimension, read_head()返回的维度
返回：
文件头的长度
'''
def get_head_length(dimension):
    return 4*len(dimension)+4

In [4]:
'''
功能：
读出文件中的第offset张图片开始的amount张图片,mnist单张图片的数据为28*28个字节
入参：
filename, 样本图片的文件名称
head_len, 文件头长度
offset, 偏移位置，从第offset张图片开始的位置
amount, 要返回的图像数量
返回：
image_list,
'''
def read_image_vector(filename,head_len,offset,amount):
    image_mat=np.zeros((amount,IMAGE_SIZE),dtype=np.uint8)
    
    with open(filename,'rb') as pf:
        #magic_num的长度为4，dimension_cnt单个长度为4,前面的number个长度为28*28*offset	
        pf.seek(head_len+IMAGE_SIZE*offset) 
        
        for ind in range(amount):
            image = np.zeros((1,IMAGE_SIZE),dtype=np.uint8)#创建一个1，28x28的array，数据类型为uint8
            for row in range(IMAGE_SIZE):#处理28行数据，
                data = pf.read(1)#单个读出数据
                pix = int.from_bytes(data,byteorder='big')#由byte转换为int类型，
                #简单滤波，如果该位置的数值大于指定值，则表示该像素为1.因为array已经初始化为0了，如果小于该指定值，不需要变化
                if pix >10:image[0][row] = 1
            image_mat[ind,:]=image
    
    return image_mat

In [5]:
'''
功能：
读出文件中的第n张图片开始的amout个的label
入参：
filename, 样本标签的文件名称
head_len, 文件头长度
offset, 偏移位置，从第offset张图片开始的位置
amount, 要返回的图像数量
返回：
label_list，标签list
'''
def read_label_vector(filename,head_len,offset,amount):
    label_list=[]
    
    with open(filename,'rb') as pf:
        #pf 指向label的第number个数据,magic_num的长度为4，dimension_cnt单个长度为4
        pf.seek(head_len+offset) 
        
        for ind in range(amount):
            data = pf.read(1)
            label = int.from_bytes(data,byteorder='big')#由byte转换为int类型，	
            label_list.append(label)
    
    return label_list

In [6]:
'''
从文件中读offset起始位置开始读出amout个image和label。
'''
def read_image_label_vector(image_file,label_file,offset,amount):
    
    image_dim = read_head(image_file)
    label_dim = read_head(label_file)
    
    #获取样本image和label的头文件长度
    image_head_len = get_head_length(image_dim)
    label_head_len = get_head_length(label_dim)
    
    #得到image和label的向量
    image_mat = read_image_vector(image_file,image_head_len,offset,amount)
    label_list = read_label_vector(label_file,label_head_len,offset,amount)
    
    return image_mat,label_list

In [7]:
def classify(inX, dataset, labels, k):
    datasetsize = dataset.shape[0]
    #距离计算公式
    diffMat = np.tile(inX, (datasetsize, 1)) - dataset
    sqDiffMat = diffMat ** 2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances ** 0.5
    # 距离从大到小排序，返回距离的序号
    sortedDistIndicies = distances.argsort()
    # 字典
    classCount = {}
    # 前K个距离最小的
    for i in range(k):
        # sortedDistIndicies[0]返回的是距离最小的数据样本的序号
        # labels[sortedDistIndicies[0]]距离最小的数据样本的标签
        voteIlabel = labels[sortedDistIndicies[i]]
        # 若属于某类则权重加一
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    # 排序
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [8]:
if __name__ == '__main__':
    
    #保存sys.stdout，以便恢复
    __console__ = sys.stdout
    #输入要保存的文件名
    result = open('result\\result_9.txt', mode = 'w',encoding='utf-8')
    #将sys.stdout重定向到指定文件，保存输出结果
    sys.stdout = result
    train_image_file = 'MNIST_data\\train-images.idx3-ubyte'
    train_label_file = 'MNIST_data\\train-labels.idx1-ubyte'
    test_image_file = 'MNIST_data\\t10k-images.idx3-ubyte'
    test_label_file = 'MNIST_data\\t10k-labels.idx1-ubyte'
    train_image_mat, train_label_list  = read_image_label_vector(train_image_file,train_label_file,0,60000)
    test_image_mat, test_label_list  = read_image_label_vector(test_image_file,test_label_file,0,10000)
    
    #开始分类
    #修改k的值
    k = 9
    err_count = 0.0#记录错误数量
    for i in range(len(test_image_mat)):
        #利用knn算法进行分类
        class_result = classify(test_image_mat[i], train_image_mat, train_label_list, k)#计算分类结果
        print( "\n第 %d 张图片,\t分类器结果: %d,\t实际值: %d" % (i,class_result, test_label_list[i]),end=' ')
        #判断分类结果是发和标签一致
        if (class_result != test_label_list[i]): 
            print('\t分类错误！',end = ' ')
            err_count += 1.0

    print( "\n总错误数: %d" % err_count)
    print( "总错误率: %2.2f%%" % (100.0*err_count/len(test_image_mat)))
    result.close()
    #恢复sys.stdout
    sys.stdout = __console__
    print("分类已完成!")
    print( "\n总错误数: %d" % err_count)
    print( "总错误率: %2.2f%%" % (100.0*err_count/len(test_image_mat)))

分类已完成!

总错误数: 350
总错误率: 3.50%
